# TF Serving Basics

## Install docker (assumes Deep Learning VM)

```
sudo apt-get remove docker docker-engine docker.io

sudo apt-get install \
     apt-transport-https \
     ca-certificates \
     curl \
     gnupg2 \
     software-properties-common

curl -fsSL https://download.docker.com/linux/debian/gpg | sudo apt-key add -

sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/debian \
   $(lsb_release -cs) \
   stable"

sudo apt-get update

sudo apt-get install -y docker-ce

sudo usermod -a -G docker $USER

```

* **Exit your shell and relogin**

## Pull TF Serving container

```
docker pull tensorflow/serving
```

## Serving example

Once you have pulled the serving image, you can try serving an example model.

We will use a toy model called Half Plus Three, which will predict values 0.5 * x + 3 for the values we provide for prediction.

To get this model, first clone the TensorFlow Serving repo:


```
mkdir -p /tmp/tfserving
cd /tmp/tfserving
git clone https://github.com/tensorflow/serving
```

Next, run the TensorFlow Serving container pointing it to this model and opening the REST API port (8501):


```
docker run -p 8501:8501 \
-v /tmp/tfserving/serving/tensorflow_serving/servables/tensorflow/testdata/saved_model_half_plus_three:/models/half_plus_three \
-e MODEL_NAME=half_plus_three -t tensorflow/serving &
```

This will run the docker container and launch the TensorFlow Serving Model Server, bind the REST API port 8501, and map our desired model from our host to where models are expected in the container. We also pass the name of the model as an environment variable, which will be important when we query the model.

To query the model using the predict API, you can run:

```
curl -d '{"instances": [1.0, 2.0, 5.0]}' -X POST http://localhost:8601/v1/models/half_plus_three:predict
```